In [43]:
import pandas as pd
import numpy as np


from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
column_names = [i for i in range(0, 16)]

population_df = pd.DataFrame(columns=column_names)

In [71]:
# https://ecoagi.ai/de/topics/Python/python-get-all-files-in-directory

import os 
import re

list_xlsx_files = os.listdir('../data/input/population')
list_xlsx_files



for filename in list_xlsx_files: 
    print(f'test {filename}')

    # extract year and half_year
    temp = re.search(r'(\d{4})h(\d{2})', filename)
    year, half_year = temp.groups()
    print(f'{year}, {half_year}')

    file_path = f'../data/input/population/{filename}'

    raw_data = pd.read_excel('../data/input/population/SB_A01-16-00_2018h01_BE.xlsx', 'T2', header = None)

    Nan_rows = raw_data.isnull().any(axis=1)
    Non_nan_rows = ~Nan_rows

    final_rows = Non_nan_rows[Non_nan_rows].index.tolist()
    
    pop_data = raw_data.iloc[final_rows]
    pop_data.insert(0, 'year', year)
    pop_data.insert(1, 'half_year', half_year)
    print(pop_data.shape)


    pop_data.to_csv(f'../data/output/population{year}h{half_year}.csv', index = False)

test SB_A01-16-00_2019h01_BE.xlsx
2019, 01
(447, 17)
test SB_A01-16-00_2021h02_BE.xlsx
2021, 02
(447, 17)
test SB_A01-16-00_2019h02_BE.xlsx
2019, 02
(447, 17)
test SB_A01-16-00_2021h01_BE.xlsx
2021, 01
(447, 17)
test SB_A01-16-00_2022h01_BE.xlsx
2022, 01
(447, 17)
test SB_A01-16-00_2018h01_BE.xlsx
2018, 01
(447, 17)
test SB_A01-16-00_2020h02_BE.xlsx
2020, 02
(447, 17)
test SB_A01-16-00_2018h02_BE.xlsx
2018, 02
(447, 17)
test SB_A01-16-00_2020h01_BE.xlsx
2020, 01
(447, 17)
test SB_A01-16-00_2022h02_BE.xlsx
2022, 02
(447, 17)


In [45]:
pop18_1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2 Einwohnerinnen und Einwohner in Berlin am 3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bezirk\nPrognoseraum\nBezirksregion\nPlanungsraum,NaN,NaN,NaN,Ins-\ngesamt,Davon im Alter von ... bis unter ... Jahren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Und zwar,NaN
3,NaN,NaN,NaN,NaN,NaN,unter 6,6 - 15,15 - 18,18 - 27,27 - 45,45 - 55,55 - 65,65 und\nmehr,weiblich,Ausländer
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,12,30.0,43.0,13,6704,328,439,128,671,1676,1051,987,1424,3343,960
485,12,30.0,43.0,14,9714,423,557,205,830,1972,1509,1402,2816,5182,643
486,NaN,NaN,NaN,Zusammen,263920,15494,21756,7242,25905,57810,38191,35051,62471,134273,45979
487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
Nan_rows = raw_data.isnull().any(axis=1)
Non_nan_rows = ~Nan_rows



In [47]:

final_rows = Non_nan_rows[Non_nan_rows].index.tolist()

In [48]:
final_rows

pop_data = raw_data.iloc[final_rows]

column_names = raw_data.iloc[3]

In [49]:
print(column_names)
pop_data.insert(0, 'year', '2018-06-01')

pop_data.head()



0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
5          unter 6
6           6 - 15
7          15 - 18
8          18 - 27
9          27 - 45
10         45 - 55
11         55 - 65
12    65 und\nmehr
13        weiblich
14       Ausländer
Name: 3, dtype: object


,year,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
6,2018-06-01,01,1.0,11.0,01,3269,153,127,32,502,999,348,389,719,1584,1338
7,2018-06-01,01,1.0,11.0,02,185,4,9,3,15,51,36,40,27,71,61
8,2018-06-01,01,1.0,11.0,03,5073,254,329,122,578,1430,682,770,908,2336,1787
9,2018-06-01,01,1.0,11.0,04,4781,277,396,135,912,1661,629,426,345,2160,1909
10,2018-06-01,01,1.0,11.0,05,1322,83,64,13,103,455,199,185,220,602,605


In [50]:
Nan_rows


0       True
1       True
2       True
3       True
4       True
       ...  
484    False
485    False
486     True
487     True
488     True
Length: 489, dtype: bool